# Hizawye AI — Full Workflow (Colab or Kaggle)

This notebook clones the repo, sets up dependencies, runs a short session, generates reports, and verifies learning with a second model.

## 0) Environment Selector
Set `PLATFORM` to `colab` or `kaggle`.

In [3]:
PLATFORM = 'colab'  # 'colab' or 'kaggle'
REPO_URL = 'https://github.com/hizawye/hizawyeAI.git'
MODEL = 'llama3.2:3b'
RUN_SECONDS = 20  # short run for quick tests
JUDGE_MODEL = MODEL  # can set to a different model for evaluation


## 1) System Setup (Ollama + deps)
This installs Ollama and Python deps.

In [21]:
import os, subprocess, textwrap, sys

def run(cmd):
    print(f'$ {cmd}')
    try:
        # Capture both stdout and stderr for all commands for better debugging
        process = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
        if process.stdout:
            print(process.stdout.strip())
        if process.stderr:
            print(process.stderr.strip())
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e}\nStdout: {e.stdout}\nStderr: {e.stderr}")
        raise

if PLATFORM == 'colab':
    run('apt-get update -y')
    run('apt-get install -y curl')
    run('apt-get install -y zstd') # Install zstd as required by Ollama installer

    # Use the official install.sh script
    run('curl -fsSL https://ollama.com/install.sh -o install_ollama.sh')
    run('chmod +x install_ollama.sh')
    run('bash install_ollama.sh')

elif PLATFORM == 'kaggle':
    run('apt-get update -y')
    run('apt-get install -y curl')
    run('apt-get install -y zstd') # Install zstd as required by Ollama installer

    # Use the official install.sh script
    run('curl -fsSL https://ollama.com/install.sh -o install_ollama.sh')
    run('chmod +x install_ollama.sh')
    run('bash install_ollama.sh')
else:
    raise ValueError('PLATFORM must be colab or kaggle')

$ apt-get update -y
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
$ apt-get install -y curl
Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.81.0-1ubuntu1.21).
0 upgraded,

## 2) Clone Repo

In [31]:
import os, subprocess

if not os.path.exists('hizawyeAI'):
    subprocess.run(f'git clone {REPO_URL}', shell=True, check=True)
os.chdir('hizawyeAI')
print('cwd:', os.getcwd())


cwd: /content/hizawyeAI/hizawyeAI/hizawyeAI


## 3) Python Dependencies

In [32]:
import subprocess
subprocess.run('pip install -r requirements.txt', shell=True, check=True)


CompletedProcess(args='pip install -r requirements.txt', returncode=0)

## 4) Start Ollama + Pull Model

In [33]:
import subprocess, time

ollama_path = '/usr/local/bin/ollama'

# Start Ollama server in background using the full path
# Using a list for args ensures direct execution without a shell.
subprocess.Popen([ollama_path, 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5) # Give it more time to start

# Pull the model using the full path
# Using a list for arguments avoids shell parsing issues and ensures direct execution.
subprocess.run([ollama_path, 'pull', MODEL], check=True)

CompletedProcess(args=['/usr/local/bin/ollama', 'pull', 'llama3.2:3b'], returncode=0)

## 5) Initialize Mind

In [34]:
import subprocess
subprocess.run('python birth.py', shell=True, check=True)


CompletedProcess(args='python birth.py', returncode=0)

## 6) Run a Short Session

In [35]:
import subprocess
subprocess.run(f'python hizawye_ai.py & sleep {RUN_SECONDS}; pkill -INT -f hizawye_ai.py', shell=True, check=False)


CompletedProcess(args='python hizawye_ai.py & sleep 20; pkill -INT -f hizawye_ai.py', returncode=-2)

## 7) Generate Reports

In [36]:
import subprocess
subprocess.run('python analyze.py --format reports', shell=True, check=True)
subprocess.run('ls -la reports', shell=True, check=True)


CompletedProcess(args='ls -la reports', returncode=0)

## 8) Verify Learning with a Judge Model

In [26]:
import subprocess
subprocess.run(f'python evaluate_learning.py --mode both --model {JUDGE_MODEL}', shell=True, check=False)
subprocess.run('ls -la reports | tail -n 5', shell=True, check=True)


CompletedProcess(args='ls -la reports | tail -n 5', returncode=0)

## 9) View Latest Reports

In [30]:
from pathlib import Path
report = Path('reports/session_summary.md')
print(report.read_text() if report.exists() else 'No report found')


No report found
